In [1]:
%status

{'status': 'healthy',
 'startTime': 'Sun Dec 10 13:17:44 UTC 2023',
 'dbEngineVersion': '1.2.1.0.R7',
 'role': 'writer',
 'dfeQueryEngine': 'viaQueryHint',
 'gremlin': {'version': 'tinkerpop-3.6.2'},
 'sparql': {'version': 'sparql-1.1'},
 'opencypher': {'version': 'Neptune-9.0.20190305-1.0'},
 'labMode': {'ObjectIndex': 'disabled',
  'ReadWriteConflictDetection': 'enabled'},
 'features': {'SlowQueryLogs': 'disabled',
  'ResultCache': {'status': 'disabled'},
  'IAMAuthentication': 'disabled',
  'Streams': 'disabled',
  'AuditLog': 'disabled'},
 'settings': {'clusterQueryTimeoutInMs': '120000',
  'SlowQueryLogsThreshold': '5000'}}

In [2]:
%seed

Dropdown(description='Source type:', options=('', 'samples', 'custom'), style=DescriptionStyle(description_wid…

Dropdown(description='Data model:', layout=Layout(display='none', visibility='hidden'), options=('', 'property…

Dropdown(description='Language:', layout=Layout(display='none', visibility='hidden'), options=('', 'opencypher…

Dropdown(description='Language:', layout=Layout(display='none', visibility='hidden'), options=('', 'opencypher…

Dropdown(description='Data set:', layout=Layout(display='none', visibility='hidden'), options=(), style=Descri…

...

Dropdown(description='Location:', layout=Layout(display='none', visibility='hidden'), options=('Local', 'S3'),…

FileChooser(path='/home/ec2-user/SageMaker', filename='', title='', show_hidden=False, select_desc='Select', c…

Button(description='Submit', layout=Layout(visibility='hidden'), style=ButtonStyle())

Output()

Output()

In [3]:
%%oc -d code -g country -de dist
MATCH p=(a:airport)-[:route]->(b:airport)<-[:contains]-(c:country)
WHERE a.code IN ['LHR'] AND b.code IN ['AMS','JFK','NRT','PER']
RETURN p
LIMIT 4

### 参考资料
https://github.com/langchain-ai/langchain
https://python.langchain.com/docs/use_cases/graph/neptune_cypher_qa


In [4]:
!pip install langchain

In [5]:
from langchain.graphs import NeptuneGraph

#neptune host
host = "neptunedbcluster-nn9wk5uwpeyr.cluster-cj0qabvcmceu.us-west-2.neptune.amazonaws.com"
port = 8182
use_https = True

graph = NeptuneGraph(host=host, port=port, use_https=use_https)

In [6]:
graph.schema

"\n        Node properties are the following:\n        [{'properties': [{'property': 'names', 'type': 'LIST'}, {'property': 'names', 'type': 'STRING'}, {'property': 'primaryName', 'type': 'STRING'}, {'property': 'primaryName', 'type': 'LIST'}], 'labels': 'LOCATION'}, {'properties': [{'property': 'type', 'type': 'STRING'}, {'property': 'desc', 'type': 'STRING'}, {'property': 'code', 'type': 'STRING'}], 'labels': 'continent'}, {'properties': [{'property': 'type', 'type': 'STRING'}, {'property': 'desc', 'type': 'STRING'}, {'property': 'code', 'type': 'STRING'}], 'labels': 'country'}, {'properties': [{'property': 'primaryName', 'type': 'STRING'}, {'property': 'names', 'type': 'STRING'}, {'property': 'names', 'type': 'LIST'}], 'labels': 'SHELF_OFFERING'}, {'properties': [{'property': 'primaryName', 'type': 'STRING'}, {'property': 'names', 'type': 'STRING'}, {'property': 'names', 'type': 'LIST'}, {'property': 'primaryName', 'type': 'LIST'}], 'labels': 'PERSON'}, {'properties': [{'property': 

In [9]:
import os
import json
from typing import Optional
from langchain.llms.bedrock import Bedrock

# External Dependencies:
import boto3
from botocore.config import Config

def get_bedrock_client(
    assumed_role: Optional[str] = None,
    region: Optional[str] = None,
    runtime: Optional[bool] = True,
):
  
    if region is None:
        target_region = os.environ.get("AWS_REGION", os.environ.get("AWS_DEFAULT_REGION"))
    else:
        target_region = region

    print(f"Create new client\n  Using region: {target_region}")
    session_kwargs = {"region_name": target_region}
    client_kwargs = {**session_kwargs}

    profile_name = os.environ.get("AWS_PROFILE")
    if profile_name:
        print(f"  Using profile: {profile_name}")
        session_kwargs["profile_name"] = profile_name

    retry_config = Config(
        region_name=target_region,
        retries={
            "max_attempts": 10,
            "mode": "standard",
        },
    )
    session = boto3.Session(**session_kwargs)

    if assumed_role:
        print(f"  Using role: {assumed_role}", end='')
        sts = session.client("sts")
        response = sts.assume_role(
            RoleArn=str(assumed_role),
            RoleSessionName="langchain-llm-1"
        )
        print(" ... successful!")
        client_kwargs["aws_access_key_id"] = response["Credentials"]["AccessKeyId"]
        client_kwargs["aws_secret_access_key"] = response["Credentials"]["SecretAccessKey"]
        client_kwargs["aws_session_token"] = response["Credentials"]["SessionToken"]
        

    if runtime:
        service_name='bedrock-runtime'
    else:
        service_name='bedrock'

    client_kwargs["aws_access_key_id"] = os.environ.get("AWS_ACCESS_KEY_ID","")
    client_kwargs["aws_secret_access_key"] = os.environ.get("AWS_SECRET_ACCESS_KEY","")
    
    bedrock_client = session.client(
        service_name=service_name,
        config=retry_config,
        **client_kwargs
    )

    print("boto3 Bedrock client successfully created!")
    print(bedrock_client._endpoint)
    return bedrock_client


## for aksk bedrock
def get_bedrock_aksk(secret_name='chatbot_bedrock', region_name = "us-west-2"):
    # Create a Secrets Manager client
    session = boto3.session.Session()
    client = session.client(
        service_name='secretsmanager',
        region_name=region_name
    )

    try:
        get_secret_value_response = client.get_secret_value(
            SecretId=secret_name
        )
    except ClientError as e:
        # For a list of exceptions thrown, see
        # https://docs.aws.amazon.com/secretsmanager/latest/apireference/API_GetSecretValue.html
        raise e

    # Decrypts secret using the associated KMS key.
    secret = json.loads(get_secret_value_response['SecretString'])
    return secret['BEDROCK_ACCESS_KEY'],secret['BEDROCK_SECRET_KEY']

ACCESS_KEY, SECRET_KEY=get_bedrock_aksk()

#role based initial client#######
os.environ["AWS_DEFAULT_REGION"] = "us-west-2"  # E.g. "us-east-1"
os.environ["AWS_PROFILE"] = "default"
#os.environ["BEDROCK_ASSUME_ROLE"] = "arn:aws:iam::687912291502:role/service-role/AmazonSageMaker-ExecutionRole-20211013T113123"  # E.g. "arn:aws:..."
os.environ["AWS_ACCESS_KEY_ID"]=ACCESS_KEY
os.environ["AWS_SECRET_ACCESS_KEY"]=SECRET_KEY

#新boto3 sdk只能session方式初始化bedrock
boto3_bedrock = get_bedrock_client(
    #assumed_role=os.environ.get("BEDROCK_ASSUME_ROLE", None),
    region=os.environ.get("AWS_DEFAULT_REGION", None)
)

parameters_bedrock = {
    "max_tokens_to_sample": 2048,
    #"temperature": 0.5,
    "temperature": 0,
    #"top_k": 250,
    #"top_p": 1,
    "stop_sequences": ["\n\nHuman"],
}

bedrock_llm = Bedrock(model_id="anthropic.claude-v2:1", client=boto3_bedrock, model_kwargs=parameters_bedrock)


Create new client
  Using region: us-west-2
  Using profile: default
boto3 Bedrock client successfully created!
bedrock-runtime(https://bedrock-runtime.us-west-2.amazonaws.com)


In [11]:
###test the bedrock langchain integration###
bedrock_llm.predict("Human:how do you describe LLM?\n"+
           "Assistant:")

" I don't have a specific description of LLM. LLM stands for Master of Laws, an advanced law degree that one can pursue after obtaining a JD (Juris Doctor) degree. Since LLM programs come in many different focus areas from various law schools, I would need more context about which particular LLM you are asking about to provide a meaningful description. In general though, LLMs allow lawyers to gain specialized knowledge in a particular area of law."

In [12]:
from langchain.chains import NeptuneOpenCypherQAChain
from langchain.chat_models import ChatOpenAI

#llm = ChatOpenAI(temperature=0, model="gpt-4")
chain = NeptuneOpenCypherQAChain.from_llm(llm=bedrock_llm, graph=graph, verbose=True, top_K=10,
                                         return_intermediate_steps=True, return_direct=False)
#chain = NeptuneOpenCypherQAChain.from_llm(llm=llm, graph=graph)

PE words:
Generate the query in openCypher format and follow these rules:
Use only the provided relationship types and properties in the schema.
Do not include any explanations or apologies in your responses.


In [13]:
chain.run('''What is the dist from London Heathrow to Perth nonstop?''')



> Entering new NeptuneOpenCypherQAChain chain...
Generated Cypher:
MATCH (lh:airport {primaryName:"London Heathrow"})-[:route*..15]-(p:airport {primaryName:"Perth"}) 
RETURN lh.primaryName AS `From`, p.primaryName AS `To`, REDUCE(dist = 0, r in RELATIONSHIPS(lh)-[]->() | dist + r.dist) AS `Distance`



NeptuneQueryException: {'message': 'An error occurred while executing the query.', 'details': "An error occurred (MalformedQueryException) when calling the ExecuteOpenCypherQuery operation: Invalid input '>' (line 2, column 100 (offset: 199))"}

In [14]:
chain.run('''Can I fly from Austin to Sydney with no stops?''')



> Entering new NeptuneOpenCypherQAChain chain...
Generated Cypher:
MATCH p=shortestPath((austin:airport {primaryName:"Austin"})-[:route*]->(sydney:airport {primaryName:"Sydney"}))
WHERE NONE (x in RELATIONSHIPS(p) WHERE x.dist > 0)
RETURN EXISTS(p)



NeptuneQueryException: {'message': 'An error occurred while executing the query.', 'details': 'An error occurred (MalformedQueryException) when calling the ExecuteOpenCypherQuery operation: Argument to EXISTS(...) is not a property or pattern (line 3, column 15 (offset: 179))'}

In [15]:
chain.run('''How many outgoing routes does the Austin (AUS) airport have?''')



> Entering new NeptuneOpenCypherQAChain chain...
Generated Cypher:
MATCH (a:airport {primaryName:"Austin"})-[:route]->(b)
RETURN count(b) AS numRoutes

Full Context:
{'ResponseMetadata': {'HTTPStatusCode': 200, 'HTTPHeaders': {'transfer-encoding': 'chunked', 'content-type': 'application/json;charset=UTF-8'}, 'RetryAttempts': 0}, 'results': [{'numRoutes': 0}]}

> Finished chain.


" Unfortunately, I do not have enough information to definitively state how many outgoing routes the Austin airport has. The information provided indicates there are 0 routes, but does not specify if this refers to outgoing routes from the Austin airport specifically. Without more context or details, I cannot reliably answer the question about the number of outgoing routes from the Austin airport. I apologize that I don't have sufficient information to provide a helpful answer. Please let me know if you can provide any additional details that might help me understand the context better."

In [16]:
chain.run('''Find the top 10 airports sorted by the number of outgoing routes?
Return just the airport code and the number of routes''')



> Entering new NeptuneOpenCypherQAChain chain...
Generated Cypher:
 MATCH (a:airport)-[:route]->() 
WITH a, COUNT(*) AS routes
RETURN a.code, routes
ORDER BY routes DESC
LIMIT 10

Full Context:
{'ResponseMetadata': {'HTTPStatusCode': 200, 'HTTPHeaders': {'transfer-encoding': 'chunked', 'content-type': 'application/json;charset=UTF-8'}, 'RetryAttempts': 0}, 'results': [{'a.code': 'IST', 'routes': 307}, {'a.code': 'FRA', 'routes': 307}, {'a.code': 'CDG', 'routes': 293}, {'a.code': 'AMS', 'routes': 282}, {'a.code': 'MUC', 'routes': 270}, {'a.code': 'ORD', 'routes': 264}, {'a.code': 'DFW', 'routes': 251}, {'a.code': 'PEK', 'routes': 248}, {'a.code': 'DXB', 'routes': 247}, {'a.code': 'ATL', 'routes': 242}]}

> Finished chain.


' Based on the provided information, the top 10 airports sorted by the number of outgoing routes are:\n\nIST with 307 routes\nFRA with 307 routes  \nCDG with 293 routes\nAMS with 282 routes\nMUC with 270 routes\nORD with 264 routes\nDFW with 251 routes\nPEK with 248 routes \nDXB with 247 routes\nATL with 242 routes'